In [ ]:
!pip install beir
!pip install elasticsearch
!pip install -U sentence-transformers
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/11

In [ ]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.retrieval import models

import pathlib, os, string
from tqdm.notebook import tqdm

import spacy

import time
import numpy as np
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp
import string

from sentence_transformers import SentenceTransformer

import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Application running on {device}")

/usr/local/lib/python3.9/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Application running on cuda


## Download and setup the Elasticsearch instance

In [ ]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512 

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [ ]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
for i in tqdm(range(20), desc = 'Let the Elasticsearch instance start'): time.sleep(1)

Let the Elasticsearch instance start:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
%%bash

ps -ef | grep elasticsearch

root        1679    1673  0 11:42 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon      1681    1679 79 11:42 ?        00:00:15 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-10866583681410291828 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:fileco

In [ ]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "ac2bb6325999",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "RFP_Ar6-TBaIDThjR9uPlA",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


# Data Loading

In [ ]:
def download_dataset(dataset):
  data_path = f'datasets/{dataset}'
  url = f'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{dataset}.zip'
  out_dir = os.path.join(os.getcwd(), 'datasets')
  data_path = util.download_and_unzip(url, out_dir)
  print(f'Dataset downloaded here: {data_path}')
  return GenericDataLoader(data_path).load(split="test")

dataset = 'scifact'
corpus, queries, qrels = download_dataset(dataset)

/content/datasets/scifact.zip:   0%|          | 0.00/2.69M [00:00<?, ?iB/s]

Dataset downloaded here: /content/datasets/scifact


  0%|          | 0/5183 [00:00<?, ?it/s]

In [ ]:
def embeddings(model, corpus, queries, qrels):
  retriever = EvaluateRetrieval(model, score_function='dot')

  #### Retrieve dense results (format of results is identical to qrels)
  results = retriever.retrieve(corpus, queries)

  ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
  return results, retriever.k_values, {'ndcg': ndcg, 'map': _map, 'recall': recall, 'precision': precision}

In [ ]:
def print_res(score_dict):
  for score, res in score_dict.items():
    print(score)
    for k, r in res.items():
      print(f'\t{k}\t->\t{r}')
    print('\n')

# Sparse Embeddings with BM25

In [ ]:
nlp = spacy.load('en_core_web_sm')
stopwords = nlp.Defaults.stop_words
clean_tokens = lambda tokens : ' '.join([token.lemma_.lower() if token not in stopwords and not token.is_punct else '' for token in tokens])

In [ ]:
def pre_process(elem_to_preprocess):
  key, val = elem_to_preprocess
  if type(val) is dict:
    return key, {
        'title': clean_tokens(nlp(val['title'])),
        'text': clean_tokens(nlp(val['text']))
    }
  else: return key, clean_tokens(nlp(val))

def query_documents_preprocessing(documents, queries):
  new_queries = {}
  new_documents = {}

  for text, iter, res in zip(('Documents', 'Queries'), (documents, queries), (new_documents, new_queries)):
    with ProcessPoolExecutor(max_workers=mp.cpu_count()) as executor:
      for id, query_doc in list(tqdm(executor.map(pre_process, iter.items()), total=len(iter.items()), desc=f'{text} Pre-Processing')):
        res[id] = query_doc

  return new_documents, new_queries
  
processed_corpus, processed_queries = query_documents_preprocessing(corpus, queries)

Documents Pre-Processing:   0%|          | 0/5183 [00:00<?, ?it/s]

Queries Pre-Processing:   0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
def sparse_embeddings_bm25(corpus, queries):
  hostname = 'localhost' 
  index_name = 'scifact' 
  initialize = True # True, will delete existing index with same name and reindex all documents

  model = BM25(index_name=index_name, hostname=hostname, initialize=initialize)
  return embeddings(model, corpus, queries, qrels)

sparse_vector, sparse_k_values, sparse_score_dict = sparse_embeddings_bm25(processed_corpus, processed_queries)
print(f'\nSparse retrieved evaluation for k in: {sparse_k_values}')
print_res(sparse_score_dict)

que: 100%|██████████| 3/3 [00:14<00:00,  4.73s/it]


Sparse retrieved evaluation for k in: [1, 3, 5, 10, 100, 1000]
ndcg
	NDCG@1	->	0.58
	NDCG@3	->	0.64035
	NDCG@5	->	0.67004
	NDCG@10	->	0.69265
	NDCG@100	->	0.71437
	NDCG@1000	->	0.72303


map
	MAP@1	->	0.55928
	MAP@3	->	0.61821
	MAP@5	->	0.63546
	MAP@10	->	0.64661
	MAP@100	->	0.65181
	MAP@1000	->	0.65219


recall
	Recall@1	->	0.55928
	Recall@3	->	0.68261
	Recall@5	->	0.75456
	Recall@10	->	0.81867
	Recall@100	->	0.91422
	Recall@1000	->	0.98


precision
	P@1	->	0.58
	P@3	->	0.24222
	P@5	->	0.16333
	P@10	->	0.09033
	P@100	->	0.01033
	P@1000	->	0.00111




# Dense Embeddings with SentenceBert and all-MiniLM-L6-v2

In [ ]:
def dense_embeddings_sbert(corpus, queries):
  model = DRES(models.SentenceBERT('all-MiniLM-L6-v2'), batch_size=16)
  return embeddings(model, corpus, queries, qrels)

dense_vector, dense_k_values, dense_score_dict = dense_embeddings_sbert(corpus, queries)
print(f'\nDense etrieved evaluation for k in: {dense_k_values}')
print_res(dense_score_dict)

Batches:   0%|          | 0/19 [00:00<?, ?it/s]

Batches:   0%|          | 0/324 [00:00<?, ?it/s]


Dense etrieved evaluation for k in: [1, 3, 5, 10, 100, 1000]
ndcg
	NDCG@1	->	0.50333
	NDCG@3	->	0.59673
	NDCG@5	->	0.62928
	NDCG@10	->	0.64508
	NDCG@100	->	0.67665
	NDCG@1000	->	0.68552


map
	MAP@1	->	0.48233
	MAP@3	->	0.56564
	MAP@5	->	0.58826
	MAP@10	->	0.59593
	MAP@100	->	0.60307
	MAP@1000	->	0.60343


recall
	Recall@1	->	0.48233
	Recall@3	->	0.66033
	Recall@5	->	0.73794
	Recall@10	->	0.78333
	Recall@100	->	0.925
	Recall@1000	->	0.99333


precision
	P@1	->	0.50333
	P@3	->	0.23778
	P@5	->	0.164
	P@10	->	0.08833
	P@100	->	0.01053
	P@1000	->	0.00112




# Merging Sparse and Dense Embeddings obtaining the groud truth

In [ ]:
def ground_truth_merging(sparse_vector, dense_vector):
  merged_results = {}
  k_values = [1, 3, 5, 10, 100, 1000]

  for (query, sparse_dic), (_, dense_dic) in tqdm(zip(sparse_vector.items(), dense_vector.items()), total=len(sparse_vector.items()), desc='Obtaining the merged vector'):
    merged_results[query] = {doc_id: sparse_dic.get(doc_id, 0) + dense_dic.get(doc_id, 0) for doc_id in set(sparse_dic) | set(dense_dic)}

  ndcg, _map, recall, precision = EvaluateRetrieval.evaluate(qrels, merged_results, k_values=k_values)
  return merged_results, k_values, {'ndcg': ndcg, 'map': _map, 'recall': recall, 'precision': precision}

ground_truth_vector, ground_truth_k_values, ground_truth_scores_dict = ground_truth_merging(sparse_vector, dense_vector)
print(f'\nMerged retrieved evaluation for k in: {ground_truth_k_values}')
print_res(ground_truth_scores_dict)

Obtaining the merged vector:   0%|          | 0/300 [00:00<?, ?it/s]


Merged retrieved evaluation for k in: [1, 3, 5, 10, 100, 1000]
ndcg
	NDCG@1	->	0.58
	NDCG@3	->	0.64578
	NDCG@5	->	0.67384
	NDCG@10	->	0.69424
	NDCG@100	->	0.71708
	NDCG@1000	->	0.72506


map
	MAP@1	->	0.55928
	MAP@3	->	0.6221
	MAP@5	->	0.63827
	MAP@10	->	0.6485
	MAP@100	->	0.65402
	MAP@1000	->	0.65437


recall
	Recall@1	->	0.55928
	Recall@3	->	0.69261
	Recall@5	->	0.76122
	Recall@10	->	0.81867
	Recall@100	->	0.91922
	Recall@1000	->	0.98


precision
	P@1	->	0.58
	P@3	->	0.24556
	P@5	->	0.16467
	P@10	->	0.09033
	P@100	->	0.0104
	P@1000	->	0.00111




In [ ]:
def merging_top_k(sparse_vector, dense_vector, top_k=[1, 3, 5, 10, 100, 1000]):
  merged_top_k = {}
  for k in top_k:

    merged_top_k[k] = {}

    for (query, sparse_dic), (_, dense_dic) in tqdm(zip(sparse_vector.items(), dense_vector.items()), total=len(sparse_vector.items()), desc=f'Retriving the top {k}'):
      
      merged_top_k[k][query] = {}

      sorted_sparse_score = dict(sorted(sparse_dic.items(), key=lambda item: item[1], reverse=True)[0:k])
      sorted_dense_score = dict(sorted(dense_dic.items(), key=lambda item: item[1], reverse=True)[0:k])

      merged_top_k[k][query] = sorted_sparse_score | sorted_dense_score
  
  return merged_top_k
        
merged_top_k_vector = merging_top_k(sparse_vector, dense_vector)

Retriving the top 1:   0%|          | 0/300 [00:00<?, ?it/s]

Retriving the top 3:   0%|          | 0/300 [00:00<?, ?it/s]

Retriving the top 5:   0%|          | 0/300 [00:00<?, ?it/s]

Retriving the top 10:   0%|          | 0/300 [00:00<?, ?it/s]

Retriving the top 100:   0%|          | 0/300 [00:00<?, ?it/s]

Retriving the top 1000:   0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
def evaluate_top_k(merged_top_k_vector, ground_truth_vector):
  k_values = [1, 3, 5, 10, 100, 1000]

  evaluate_top_k_results = {}

  for k, query_doc_scores in tqdm(merged_top_k_vector.items(), total=len(merged_top_k_vector.items()), desc='Evaluating the retreived top K among the sparse and dense embeddings'):
    ndcg, _map, recall, precision = EvaluateRetrieval.evaluate(ground_truth_vector, query_doc_scores, k_values=k_values)
    evaluate_top_k_results[k] = {'ndcg': ndcg, 'map': _map, 'recall': recall, 'precision': precision}
  
  return evaluate_top_k_results

results_evaluate_top_k = evaluate_top_k(merged_top_k_vector, qrels)

for k, result in results_evaluate_top_k.items():
  print(f'------------------ TOP {k} results ------------------')
  print_res(result)
  print('\n')

[1, 3, 5, 10, 100, 1000]


Evaluating the retreived top K among the sparse and dense embeddings:   0%|          | 0/6 [00:00<?, ?it/s]

------------------ TOP 1 results ------------------
ndcg
	NDCG@1	->	0.58
	NDCG@3	->	0.602
	NDCG@5	->	0.6007
	NDCG@10	->	0.6007
	NDCG@100	->	0.6007
	NDCG@1000	->	0.6007


map
	MAP@1	->	0.55928
	MAP@3	->	0.58925
	MAP@5	->	0.58925
	MAP@10	->	0.58925
	MAP@100	->	0.58925
	MAP@1000	->	0.58925


recall
	Recall@1	->	0.55928
	Recall@3	->	0.6155
	Recall@5	->	0.6155
	Recall@10	->	0.6155
	Recall@100	->	0.6155
	Recall@1000	->	0.6155


precision
	P@1	->	0.58
	P@3	->	0.21667
	P@5	->	0.13
	P@10	->	0.065
	P@100	->	0.0065
	P@1000	->	0.00065




------------------ TOP 3 results ------------------
ndcg
	NDCG@1	->	0.12
	NDCG@3	->	0.40124
	NDCG@5	->	0.45452
	NDCG@10	->	0.4577
	NDCG@100	->	0.4577
	NDCG@1000	->	0.4577


map
	MAP@1	->	0.10094
	MAP@3	->	0.3177
	MAP@5	->	0.34954
	MAP@10	->	0.35109
	MAP@100	->	0.35109
	MAP@1000	->	0.35109


recall
	Recall@1	->	0.10094
	Recall@3	->	0.62761
	Recall@5	->	0.75328
	Recall@10	->	0.76128
	Recall@100	->	0.76128
	Recall@1000	->	0.76128


precision
	P@1	->	0.12
	P@3	->	0.2